# Model Evaluation

### Import Libraries & Load Data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from prophet import Prophet

from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

import xgboost as xgb

# Load cleaned data
data = pd.read_csv("../input/named_data_cleaned.csv")
data['datum'] = pd.to_datetime(data['datum'])
data = data.set_index('datum')

# List of all drug categories
drug_cols = [
    "Antiinflammatory", "Antirheumatic", "Analgesics", 
    "Antipyretics", "Psycholeptics", "Sedatives", 
    "Bronchodilators", "Antihistamines"
]

### Train-Test Split

In [4]:
train_size = 0.8

def train_test_split_series(series):
    n = len(series)
    train = series[:int(n*train_size)]
    test = series[int(n*train_size):]
    return train, test

### Evaluation Metrics

In [5]:
def evaluate_forecast(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100
    return mse, mae, mape

### ARIMA & SARIMA Forecasting

In [7]:
results = []

for col in drug_cols:
    series = data[col].dropna()
    train, test = train_test_split_series(series)
    
    # ARIMA (p,d,q) -> can use (1,1,1) as simple choice
    arima_model = ARIMA(train, order=(1,1,1)).fit()
    arima_pred = arima_model.forecast(len(test))
    arima_mse, arima_mae, arima_mape = evaluate_forecast(test, arima_pred)
    
    # SARIMA (seasonal) -> simple choice: (1,1,1)x(1,1,1,12)
    sarima_model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,12)).fit(disp=False)
    sarima_pred = sarima_model.forecast(len(test))
    sarima_mse, sarima_mae, sarima_mape = evaluate_forecast(test, sarima_pred)
    
    results.append({
        "Drug": col,
        "ARIMA_MSE": arima_mse, "ARIMA_MAE": arima_mae, "ARIMA_MAPE": arima_mape,
        "SARIMA_MSE": sarima_mse, "SARIMA_MAE": sarima_mae, "SARIMA_MAPE": sarima_mape
    })

/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWa

### Holt-Winters Forecasting

In [8]:
for col in drug_cols:
    series = data[col].dropna()
    train, test = train_test_split_series(series)
    
    hw_model = ExponentialSmoothing(train, seasonal='add', seasonal_periods=12).fit()
    hw_pred = hw_model.forecast(len(test))
    hw_mse, hw_mae, hw_mape = evaluate_forecast(test, hw_pred)
    
    # Add to results
    for r in results:
        if r['Drug'] == col:
            r["HW_MSE"] = hw_mse
            r["HW_MAE"] = hw_mae
            r["HW_MAPE"] = hw_mape

/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-SUN will be used.
  self._init_dates(dates, freq)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWa

### Prophet Forecasting

In [9]:
for col in drug_cols:
    series = data[col].dropna()
    train, test = train_test_split_series(series)
    
    df_train = train.reset_index().rename(columns={'datum':'ds', col:'y'})
    df_test = test.reset_index().rename(columns={'datum':'ds', col:'y'})
    
    model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
    model.fit(df_train)
    future = model.make_future_dataframe(periods=len(test), freq='D')
    forecast = model.predict(future)
    prophet_pred = forecast['yhat'][-len(test):].values
    
    mse, mae, mape = evaluate_forecast(test.values, prophet_pred)
    
    for r in results:
        if r['Drug'] == col:
            r["Prophet_MSE"] = mse
            r["Prophet_MAE"] = mae
            r["Prophet_MAPE"] = mape

23:16:57 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
/var/folders/vp/2_mwt2_577d0yzkwp961ny140000gn/T/ipykernel_10698/467812333.py:4: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100
23:16:58 - cmdstanpy - INFO - Chain [1] start processing
23:16:58 - cmdstanpy - INFO - Chain [1] done processing
23:16:58 - cmdstan

### LSTM Forecasting

In [10]:
scaler = MinMaxScaler()

for col in drug_cols:
    series = data[col].dropna().values.reshape(-1,1)
    series_scaled = scaler.fit_transform(series)
    
    train_len = int(len(series_scaled)*train_size)
    train_scaled = series_scaled[:train_len]
    test_scaled = series_scaled[train_len:]
    
    # Create sequences
    def create_seq(data, n_steps=5):
        X, y = [], []
        for i in range(len(data)-n_steps):
            X.append(data[i:i+n_steps])
            y.append(data[i+n_steps])
        return np.array(X), np.array(y)
    
    n_steps = 5
    X_train, y_train = create_seq(train_scaled, n_steps)
    X_test, y_test = create_seq(series_scaled[train_len-n_steps:], n_steps)
    
    # LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps,1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=50, verbose=0)
    
    y_pred_scaled = model.predict(X_test, verbose=0)
    y_pred = scaler.inverse_transform(y_pred_scaled)
    y_true = scaler.inverse_transform(y_test)
    
    mse, mae, mape = evaluate_forecast(y_true, y_pred)
    
    for r in results:
        if r['Drug'] == col:
            r["LSTM_MSE"] = mse
            r["LSTM_MAE"] = mae
            r["LSTM_MAPE"] = mape

/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/var/folders/vp/2_mwt2_577d0yzkwp961ny140000gn/T/ipykernel_10698/467812333.py:4: Runtime

### XGBoost Forecasting

In [11]:
for col in drug_cols:
    series = data[col].dropna().values
    n_steps = 5
    
    # Prepare sequences
    X, y = [], []
    for i in range(len(series)-n_steps):
        X.append(series[i:i+n_steps])
        y.append(series[i+n_steps])
    X, y = np.array(X), np.array(y)
    
    train_size_xgb = int(len(X)*train_size)
    X_train, X_test = X[:train_size_xgb], X[train_size_xgb:]
    y_train, y_test = y[:train_size_xgb], y[train_size_xgb:]
    
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse, mae, mape = evaluate_forecast(y_test, y_pred)
    
    for r in results:
        if r['Drug'] == col:
            r["XGBoost_MSE"] = mse
            r["XGBoost_MAE"] = mae
            r["XGBoost_MAPE"] = mape

/var/folders/vp/2_mwt2_577d0yzkwp961ny140000gn/T/ipykernel_10698/467812333.py:4: RuntimeWarning: divide by zero encountered in divide
  mape = np.mean(np.abs((y_true - y_pred)/y_true)) * 100


### Summary Table & Best Method Selection

In [12]:
summary_df = pd.DataFrame(results)

# Determine best method by MAPE (lower is better)
best_methods = []
for idx, row in summary_df.iterrows():
    metrics = {
        "ARIMA": row["ARIMA_MAPE"],
        "SARIMA": row["SARIMA_MAPE"],
        "HW": row["HW_MAPE"],
        "Prophet": row["Prophet_MAPE"],
        "LSTM": row["LSTM_MAPE"],
        "XGBoost": row["XGBoost_MAPE"]
    }
    best = min(metrics, key=metrics.get)
    best_methods.append(best)

summary_df["Best_Method"] = best_methods

# Display summary
summary_df.style.background_gradient(cmap='coolwarm')

/opt/anaconda3/envs/hack_env/lib/python3.10/site-packages/pandas/io/formats/style.py:3811: RuntimeWarning: invalid value encountered in scalar multiply
  norm = _matplotlib.colors.Normalize(smin - (rng * low), smax + (rng * high))


,Drug,ARIMA_MSE,ARIMA_MAE,ARIMA_MAPE,SARIMA_MSE,SARIMA_MAE,SARIMA_MAPE,HW_MSE,HW_MAE,HW_MAPE,Prophet_MSE,Prophet_MAE,Prophet_MAPE,LSTM_MSE,LSTM_MAE,LSTM_MAPE,XGBoost_MSE,XGBoost_MAE,XGBoost_MAPE,Best_Method
0,Antiinflammatory,66.923673,6.006634,20.237283,81.486470,6.703987,20.962853,70.468575,6.079790,20.047967,71.703178,6.248943,21.957851,68.148252,5.945509,21.712931,89.661700,7.298943,24.954718,HW
1,Antirheumatic,86.462747,6.972019,30.875171,82.667782,6.702777,28.614158,84.193288,6.809657,29.251936,78.859305,6.912064,30.800495,81.956677,6.996001,32.459435,84.788450,7.132083,32.985614,SARIMA
2,Analgesics,29.260181,4.243735,27.902646,38.681115,4.948318,29.145077,31.362952,4.356863,29.771188,44.606154,5.240900,29.311606,40.206297,5.209359,35.836912,41.668510,4.918521,32.180449,ARIMA
3,Antipyretics,11560.296770,81.034261,32.598137,14016.320721,93.090021,37.928530,12449.523270,84.974670,34.186613,14962.146022,104.873687,54.286985,2718.584482,38.609385,20.268871,3680.267914,44.801651,22.681536,LSTM
4,Psycholeptics,156.250899,10.255762,20.338141,205.087521,11.586764,23.442216,208.853482,11.673273,23.705521,128.830647,9.423301,17.287147,136.907400,9.266721,17.842368,270.947641,12.595473,24.010549,Prophet
5,Sedatives,7.855785,2.278488,inf,9.059249,2.358259,inf,8.673919,2.314824,inf,10.421832,2.675032,inf,7.680158,2.235172,inf,11.761895,2.759463,inf,ARIMA
6,Bronchodilators,2356.990049,39.054487,62.461410,2317.846335,38.667807,63.101706,2235.078214,37.550735,60.591978,1731.357853,34.285337,77.588218,721.326790,19.859473,42.726582,903.242395,21.750309,45.587123,LSTM
7,Antihistamines,151.355074,9.706446,98.138286,142.975436,9.082191,90.079430,142.897710,9.214434,90.893415,184.699787,10.835405,79.984443,74.632206,6.651124,62.774266,92.255333,7.610891,63.532655,LSTM
